# September 15, 2022: FC matrices: Emoprox2 trial level responses: Mashid ROIs:

In [1]:
import os
import sys
from os.path import join as pjoin

import numpy as np
import pandas as pd
import pickle, random
from tqdm import tqdm
from scipy.stats import (norm, zscore, permutation_test)
from itertools import combinations

# ISC
from brainiak.isc import (
    isc, isfc, bootstrap_isc, compute_summary_statistic, squareform_isfc, compute_correlation,
    _check_timeseries_input, _check_targets_input, _threshold_nans
)
from statsmodels.stats.multitest import multipletests
from scipy.spatial.distance import squareform
from brainiak.fcma.util import compute_correlation

# network analyses
import networkx as nx

# plotting
import matplotlib.pyplot as plt
plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr #CITE ITS PAPER IN YOUR MANUSCRIPT

# main dirs
proj_dir = pjoin(os.environ['HOME'], 'network_dynamics')
results_dir = f"{proj_dir}/results"
month_dir = f"{proj_dir}/nb/sep22"

# folders
sys.path.insert(0, proj_dir)
import helpers.dataset_utils as dataset_utils
import helpers.isc_utils as isc_utils
import helpers.fc_utils as fc_utils

In [2]:
class ARGS(): pass
args = ARGS()

args.APPR = 1.0
args.RETR = -1.0
args.LABEL_NAMES = ['RETR', 'APPR']
args.LABELS = [args.RETR, args.APPR]

# arbitrary threshold on proximity
args.near_miss_thresh = 0.7
args.near_miss_width = 8

set_name = 'mashid'

args.main_data_path = (
    f"{proj_dir}/data/emoprox2/data/rois_{set_name}"
)
# os.system(f"mkdir -p {main_output_path}")

args.SEED = 74
np.random.seed(args.SEED)

# ROIs
roi_set_file = f"{proj_dir}/data/rois/{set_name}/roi_set_{set_name}.csv"
args.roi_set_df = pd.read_csv(roi_set_file)
args.roi_set_df = args.roi_set_df.sort_values(by=['network'])
# display(args.roi_set_df)

args.roi_names = args.roi_set_df['roi_name'].values
args.nw_names = args.roi_set_df['network'].values
args.roi_idxs = args.roi_set_df.index.values
args.num_rois = len(args.roi_idxs)

# trial level responses
with open(
    f"/home/joyneelm/FORAGING/local/notebooks/dynamical_systems_emoprox/emoprox_motion_parameters.pkl", 
    'rb'
) as f:
    motion_df = pickle.load(f)

args.trial_option = 'trial'

args.subjects = motion_df.pid.unique()
random.Random(args.SEED).shuffle(args.subjects)
args.explor_subjects = args.subjects[ : round(0.25 * len(args.subjects))]

X = dataset_utils.get_emo2_trial_level_responses(
    args, 
    motion_df, 
    args.explor_subjects
)

# change roi ordering
for idx_label, label in enumerate(args.LABELS):
    for idx_subj in range(len(X[idx_label])):
        X[idx_label][idx_subj] = X[idx_label][idx_subj][:, :, args.roi_idxs]

100%|██████████| 30/30 [00:00<00:00, 37.22it/s]

RETR : maxTRs 6
APPR : maxTRs 7
